In [29]:
import os
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import timm
from torchvision import datasets, transforms, models

from tqdm.notebook import tqdm
from baseline_model import (
	set_seed,
	get_flowers_dataloaders,
	create_baseline_model,
	train_epoch,
	validate,
)
import pytorch_lightning as pl

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed()
train_loader, val_loader, test_loader = get_flowers_dataloaders(batch_size=32)
print(device)

Seed set to 42


cuda


# Different Depths

In [31]:
class EarlyStopper:
	def __init__(self, patience=10, min_delta=0):
		self.patience = patience
		self.min_delta = min_delta
		self.counter = 0
		self.min_validation_loss = np.inf

	def __call__(self, validation_loss):
		if validation_loss < self.min_validation_loss:
			self.min_validation_loss = validation_loss
			self.counter = 0
		elif validation_loss > (self.min_validation_loss + self.min_delta):
			self.counter += 1
			if self.counter >= self.patience:
				return True
		return False

In [32]:
def get_resnet_performance(model_name, num_epochs = 30, batch_size = 32, lr = 0.001, weight_decay = 1e-4, scheduler_step_size = 10, scheduler_gamma = 0.1):
	history = {
		'train_loss': [], 'train_acc': [],
		'val_loss': [], 'val_acc': [],
	}

	# Best model tracking
	best_val_acc = 0.0
	best_model_state = None

	early_stop = EarlyStopper()
	
	model = create_baseline_model(num_classes=102, pretrained=True, model_name=model_name)
	model = model.to(device)
	criterion = nn.CrossEntropyLoss().to(device)
	optimizer = optim.AdamW(
		model.parameters(),
		lr=lr,
		weight_decay=weight_decay,
	)

	# Learning rate scheduler
	scheduler = optim.lr_scheduler.StepLR(
		optimizer,
		step_size=scheduler_step_size,
		gamma=scheduler_gamma,
	)
	assert num_epochs > 0, "Num epochs must be more than 0!"

	for epoch in tqdm(range(num_epochs), f"Training Epoch ({model_name})", unit="epoch"):
		current_lr = scheduler.get_last_lr()[0]
		train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
		val_loss, val_acc = validate(model, val_loader, criterion, device)
		scheduler.step()
		history['train_loss'].append(train_loss)
		history['train_acc'].append(train_acc)
		history['val_loss'].append(val_loss)
		history['val_acc'].append(val_acc)
		if val_acc > best_val_acc:
			best_val_acc = val_acc
			best_model_state = model.state_dict().copy()
		if early_stop(val_loss):
			break
		
		tqdm.write(
			f"Epoch [{epoch+1}/{num_epochs}] "
			f"LR: {current_lr:.6f} | "
			f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%"
			f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%"
			f"Test Loss: {test_loss:.4f}, Val Acc: {test_acc:.2f}%"
		)
	test_loss, test_acc = validate(model, test_loader, criterion, device, valid_or_test = "test")
	history['test_loss'] = test_loss
	history['test_acc'] = test_acc
	assert best_val_acc > 0.0 and best_model_state is not None, "The model validation acc = 0!" # my pylance can't deduce that best_model state != 0 based on best_val_acc != 0 :(
	model.load_state_dict(best_model_state)
	print(f"\nBest Validation Accuracy: {best_val_acc:.2f}%")
	num_params = sum(p.numel() for p in model.parameters())
	return model, history, num_params


In [38]:
# List all available ResNet models
resnet_models = timm.list_models('resnet*')
print("ResNet models:")
print(resnet_models)

# List all available Vision Transformer (ViT) models
vit_models = timm.list_models('vit*')
print("\nViT models:")
print(vit_models)


ResNet models:
['resnet10t', 'resnet14t', 'resnet18', 'resnet18d', 'resnet26', 'resnet26d', 'resnet26t', 'resnet32ts', 'resnet33ts', 'resnet34', 'resnet34d', 'resnet50', 'resnet50_clip', 'resnet50_clip_gap', 'resnet50_gn', 'resnet50_mlp', 'resnet50c', 'resnet50d', 'resnet50s', 'resnet50t', 'resnet50x4_clip', 'resnet50x4_clip_gap', 'resnet50x16_clip', 'resnet50x16_clip_gap', 'resnet50x64_clip', 'resnet50x64_clip_gap', 'resnet51q', 'resnet61q', 'resnet101', 'resnet101_clip', 'resnet101_clip_gap', 'resnet101c', 'resnet101d', 'resnet101s', 'resnet152', 'resnet152c', 'resnet152d', 'resnet152s', 'resnet200', 'resnet200d', 'resnetaa34d', 'resnetaa50', 'resnetaa50d', 'resnetaa101d', 'resnetblur18', 'resnetblur50', 'resnetblur50d', 'resnetblur101d', 'resnetrs50', 'resnetrs101', 'resnetrs152', 'resnetrs200', 'resnetrs270', 'resnetrs350', 'resnetrs420', 'resnetv2_18', 'resnetv2_18d', 'resnetv2_34', 'resnetv2_34d', 'resnetv2_50', 'resnetv2_50d', 'resnetv2_50d_evos', 'resnetv2_50d_frn', 'resnetv2_5

In [ ]:
def create_vit_model(num_classes=102, pretrained=True, model_name="vit_base_patch16_224"):
	model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)
	return model

def get_vit_performance(model_name="vit_b_16", num_epochs=30, batch_size=32, lr=0.001,
			  weight_decay=1e-4, scheduler_step_size=10, scheduler_gamma=0.1):
	history = {
		'train_loss': [], 'train_acc': [],
		'val_loss': [], 'val_acc': [],
		'test_loss': [], 'test_acc': []
	}

	best_val_acc = 0.0
	best_model_state = None
	early_stop = EarlyStopper()

	# Create model
	model = create_vit_model(num_classes=102, pretrained=True, model_name=model_name)
	model = model.to(device)

	criterion = nn.CrossEntropyLoss().to(device)
	optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
	scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)

	assert num_epochs > 0, "Num epochs must be more than 0!"

	for epoch in tqdm(range(num_epochs), desc=f"Training Epoch ({model_name})", unit="epoch"):
		current_lr = scheduler.get_last_lr()[0]

		# Train / Validate / Test
		train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
		val_loss, val_acc = validate(model, val_loader, criterion, device)
		scheduler.step()

		# Track history
		for key, val in zip(['train_loss', 'train_acc', 'val_loss', 'val_acc'],
							[train_loss, train_acc, val_loss, val_acc]):
			history[key].append(val)

		# Save best model
		if val_acc > best_val_acc:
			best_val_acc = val_acc
			best_model_state = model.state_dict().copy()

		if early_stop(val_loss):
			tqdm.write("Early stopping triggered.")
			break

		tqdm.write(
			f"Epoch [{epoch+1}/{num_epochs}] "
			f"LR: {current_lr:.6f} | "
			f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
			f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}% | "
		)
	test_loss, test_acc = validate(model, test_loader, criterion, device)
	history["test_loss"] = test_loss
	history["test_acc"] = test_acc
	assert best_val_acc > 0.0 and best_model_state is not None, "The model validation acc = 0!" # my pylance can't deduce that best_model state != 0 based on best_val_acc != 0 :(
	model.load_state_dict(best_model_state)
	print(f"\nBest Validation Accuracy: {best_val_acc:.2f}%")
	num_params = sum(p.numel() for p in model.parameters())

	return model, history, num_params

In [ ]:
# test different depth
resnet_depth = []
resnet_test_acc = []
resnet_overfit_gap = []
resnet_histories = {}

# resnet memory scales linearly with number of layers since need to keep the output tensors in memory for backprop, it looks like my GPU can handle all of them.
# for model_name in [
# 	"resnet18",
# 	"resnet34",
# 	"resnet50",
# 	"resnet101",
# 	"resnet152",
# ]:
# 	model, history, num_params = get_resnet_performance(model_name)
# 	resnet_histories[model_name] = history
# 	resnet_histories[model_name]["num_params"] = num_params 

# ViT scales quadratically inversely with patch size. Looks like my GPU of 7.6 GB can't handle larger models
for model_name in [
	"vit_base_patch32_224",
	"vit_base_patch16_224",
	# "vit_large_patch32_224",
	# "vit_large_patch16_224",
	# "vit_huge_patch16_224",
]:
	model, history, num_params = get_vit_performance(model_name)
	resnet_histories[model_name] = history
	resnet_histories[model_name]["num_params"] = num_params 

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

In [ ]:
ResNet: resnet18, resnet34, resnet50

ViT: vit_base_patch16_224, vit_large_patch16_224

In [ ]:
for images, labels in test_loader:
	print(images.shape)
	break


torch.Size([32, 3, 224, 224])


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
ax1.plot(history['train_loss'], label='Train Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy plot
ax2.plot(history['train_acc'], label='Train Acc', linewidth=2)
ax2.plot(history['val_acc'], label='Val Acc', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# def create_vit_model(num_classes=102):

# Reduce Parameters

# Deformable Convolutions